In [48]:
#%load_ext lab_black

In [49]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandas as pd
import numpy as np
import os
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

credentials = service_account.Credentials.from_service_account_file(
    "/Users/miguelcouto/Downloads/zattoo-dataeng-e5f45785174f.json"
)

project_id = "zattoo-dataeng"
client = bigquery.Client(credentials=credentials, project=project_id)

In [50]:
sql_calcs = """
select mt.transaction_id,
       mt.zuid,
       mt.payment_method,
       mt.sku,
       mt.type_of_transaction,
       case
           when mt.country_name = 'Germany' then 'Germany'
           when mt.country_name = 'Austria' then 'Austria'
           else 'Switzerland' end                   as country_name,
       mt.new_booking_net_chf,
       mt.renewal_booking_net_chf,
       mt.new_booking_net_eur,
       mt.renewal_booking_net_eur,
       mt.transaction_date,
       mt.term_start,
       mt.term_end,
       product_service_view.zuya_account_permission as product_group_finance,
       mt.vat_eur,
       mt.vat_chf,
       mt.store_fees_chf,
       mt.store_fees_eur
from b2c_middleware.middlelayer_transactions mt
         LEFT JOIN b2c_middleware_import.product_product_view AS product_product_view
                   ON mt.sku = product_product_view.sku
                       AND DATE(mt.transaction_date) = product_product_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_subscriptionproduct_view AS product_subscriptionproduct_view
                   ON product_product_view.id = product_subscriptionproduct_view.id
                       AND
                      product_product_view.inserted_at_date = product_subscriptionproduct_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_offer_view AS product_offer_view
                   ON product_subscriptionproduct_view.offer_id = product_offer_view.id
                       AND product_subscriptionproduct_view.inserted_at_date = product_offer_view.inserted_at_date
         LEFT JOIN b2c_middleware_import.product_service_view AS product_service_view
                   ON product_offer_view.service_id = product_service_view.id
                       AND product_offer_view.inserted_at_date = product_service_view.inserted_at_date
where true
  and mt.app_shop_id = 'amazon'
  and mt.transaction_date >= '2022-02-01 00:00:00'
  and mt.transaction_date <= '2022-02-28 23:59:59'
-- NEW LINE! added on 2022-04-27
 -- and mt.type_of_transaction != 'refund'
           """

In [51]:
## prepare dataframe
df = pandas_gbq.read_gbq(sql_calcs, project_id=project_id, progress_bar_type=None)
type_trans_lst = ["new_sale", "renewal", "refund"]

In [52]:
parse_dates = ["term_start", "term_end"]
reporting_df = df[df["type_of_transaction"].isin(type_trans_lst)].copy()

for date in parse_dates:
    reporting_df[date] = pd.to_datetime(reporting_df[date]).dt.tz_convert(None)

In [53]:
reporting_month = reporting_df['transaction_date'].mode().dt.to_period('M').dt.strftime('%Y-%m')[0]

In [54]:
## calculate total_booking_net columns
reporting_df["total_booking_net_chf"] = (
        reporting_df["new_booking_net_chf"] + reporting_df["renewal_booking_net_chf"]
)
reporting_df["total_booking_net_eur"] = (
        reporting_df["new_booking_net_eur"] + reporting_df["renewal_booking_net_eur"]
)

In [55]:
## drop unnecessary columns
reporting_df.drop(
    [
        "new_booking_net_eur",
        "new_booking_net_chf",
        "renewal_booking_net_chf",
        "renewal_booking_net_eur",
    ],
    axis=1,
    inplace=True,
)

In [56]:
## calculate product_term_length_months
reporting_df["product_term_length_months"] = (
        (reporting_df["term_end"].dt.year - reporting_df["term_start"].dt.year) * 12
        + (reporting_df["term_end"].dt.month - reporting_df["term_start"].dt.month)
        + 1
)

In [57]:
reporting_df["product_term_length_months"][reporting_df["product_term_length_months"] < 0] = 0

In [58]:
## replacing product_term_length_months for exceptions where value is 2 instead of 1
shorter_subs = (reporting_df['term_end'] - reporting_df['term_start']).dt.days <= 30
shorter_subs_replacer = reporting_df[shorter_subs][
    (reporting_df["product_term_length_months"] == 2) & ((reporting_df['term_end']).dt.day == 1)].index.to_list()
reporting_df["product_term_length_months"].loc[shorter_subs_replacer] = 1

In [59]:
## reseting index
reporting_df = reporting_df.reindex(reporting_df.index.repeat(reporting_df["product_term_length_months"]))

In [60]:
## add revenue_month_number
reporting_df["revenue_month_number"] = reporting_df.groupby(["transaction_id"]).cumcount() + 1

In [61]:
## add revenue_month_date
reporting_df["revenue_month_date"] = reporting_df["term_start"].to_numpy().astype("datetime64[M]")
reporting_df["revenue_month_date"][reporting_df["type_of_transaction"] == "refund"] = (
    reporting_df["transaction_date"].to_numpy().astype("datetime64[M]")
)

reporting_df["revenue_month_date"] = reporting_df.apply(
    lambda x: x["revenue_month_date"]
              + pd.offsets.MonthEnd(x["revenue_month_number"])
              + pd.offsets.MonthBegin(-1),
    axis=1,
)

## NEW LINE! commenting out as we now have product_service_view.zuya_account_permission available
# reporting_df["product_group_finance"] = np.nan

reporting_df["product_term_length"] = reporting_df["revenue_month_date"].apply(
    lambda t: pd.Period(t, freq="S").days_in_month
)

reporting_df["product_term_length"][reporting_df["term_start"] > reporting_df["revenue_month_date"]] = (
                                                                                          reporting_df[
                                                                                              "term_start"].dt.daysinmonth -
                                                                                          reporting_df["term_start"].dt.day
                                                                                  ) + 1

In [62]:
## reseting index
reporting_df.reset_index(drop=True, inplace=True)

In [63]:
## set active_sub_month_end = 1 by default
reporting_df["active_sub_month_end"] = 1

## get last indices of each transaction_id group
last_idxs = (
        len(reporting_df)
        - np.unique(
    reporting_df.transaction_id.values[::-1],
    return_index=1,
)[1]
        - 1
)

reporting_df["active_sub_month_end"].iloc[last_idxs] = 0

## mark all one transaction subscriptions as 1
reporting_df["active_sub_month_end"][
    (reporting_df["revenue_month_number"] <= 1) & (reporting_df["product_term_length_months"] <= 1)
    ] = 1

## mark all refund transactions as -1
reporting_df["active_sub_month_end"][reporting_df["type_of_transaction"] == "refund"] = -1

## mark all subscriptions with only 1 transaction as i
reporting_df["active_sub_month_end"][reporting_df["type_of_transaction"] == "refund"] = -1

In [64]:
## active_sub_content follows the same logic as active_sub_month_end except it doesn't count the last month
reporting_df["active_sub_content"] = reporting_df.active_sub_month_end

reporting_df.loc[reporting_df.groupby('transaction_id')['active_sub_content'].tail(2).index, 'active_sub_content'] = 0

In [65]:
## fix last position of product_term_length per transaction_id
reporting_df["product_term_length"].iloc[last_idxs] = (
        reporting_df["term_end"].iloc[last_idxs] -
        reporting_df["revenue_month_date"].iloc[last_idxs]
).dt.days

In [66]:
## TEMP FIX to tackle dynamic term_end for refunds
reporting_df["product_term_length"][reporting_df["product_term_length"] < 0] = 0

In [67]:
## total_days of product_term_length per transaction_id
reporting_df["total_days"] = reporting_df.groupby("transaction_id")["product_term_length"].transform(
    "sum"
)

In [68]:
## get 12mo subs that didn't stay for 12 months for posterior treatment of exception
reporting_df['max_revenue_month_number'] = reporting_df.groupby(['transaction_id'])['revenue_month_number'].transform(max)

# reporting_df[(reporting_df.sku.str.contains('zattoo_web_hiq_swiss_12mo')) & (reporting_df.max_revenue_month_number < 13)]

In [69]:
## NEW LINE! added on 2022-04-27
## product_term_length fix for subscriptions < 31 days (1, 3, 7 days)
reporting_df['product_term_length'][reporting_df.sku.str.contains('day')] = (reporting_df.term_end - reporting_df.term_start).dt.days

In [70]:
trx_lst = reporting_df['transaction_id'][(reporting_df.sku.str.contains('day')) & (reporting_df['revenue_month_number'] > 1)].unique()

In [71]:
## total_days fix for subscriptions < 31 days (1, 3, 7 days)
reporting_df["total_days"][(reporting_df.product_term_length != reporting_df.total_days) & (reporting_df['product_term_length_months'] <= 1)] = reporting_df["product_term_length"][
    (reporting_df.product_term_length != reporting_df.total_days) & (reporting_df['product_term_length_months'] <= 1)]

In [72]:
## calculate total_revenue_net fields
reporting_df["total_revenue_net_eur"] = (
        reporting_df["total_booking_net_eur"] / reporting_df["total_days"] * reporting_df["product_term_length"]
)

reporting_df["total_revenue_net_chf"] = (
        reporting_df["total_booking_net_chf"] / reporting_df["total_days"] * reporting_df["product_term_length"]
)

## remove total_booking values from all lines of group except first
reporting_df.loc[
    reporting_df["revenue_month_number"] > 1,
    ["total_booking_net_chf", "total_booking_net_eur"],
] = 0.0

In [73]:
## remove VAT values from all lines of group except first
vat_cols = ["vat_chf", "vat_eur"]

for col in vat_cols:
    reporting_df.loc[
        reporting_df["revenue_month_number"] > 1,
        [col, "total_booking_net_eur"],
    ] = 0.0

In [74]:
## prepare df_nocalcs for free trials and full discounts
reporting_df_nocalcs = df[~df["type_of_transaction"].isin(type_trans_lst)].copy()

for date in parse_dates:
    reporting_df_nocalcs[date] = pd.to_datetime(reporting_df_nocalcs[date]).dt.tz_convert(None)

In [75]:
## extract list of transaction ids whose subscription float between one month and the other, and are 1, 3 or 7 days (== product_length < 31)
trx_lst_more_1month_subs = reporting_df['transaction_id'][
    (reporting_df["revenue_month_number"] > 1) & (reporting_df.sku.str.contains('day'))]

## this logic makes sure the days are allocated correctly for these very specific subscriptions
reporting_df['product_term_length'][
    (reporting_df.transaction_id.isin(trx_lst_more_1month_subs)) & (reporting_df.revenue_month_number == 1)] = - (
        reporting_df.term_start - reporting_df.revenue_month_date.shift(1)).dt.days

reporting_df['product_term_length'][
    (reporting_df.transaction_id.isin(trx_lst_more_1month_subs)) & (reporting_df.revenue_month_number > 1)] = (
        reporting_df.term_end - reporting_df.revenue_month_date).dt.days

In [76]:
## fix active_sub_month_end for subscriptions with only 1 revenue month and length < 30 days
reporting_df['last_day_term_end'] = (pd.to_datetime(reporting_df['term_end'], format="%d", unit='d')).dt.day
reporting_df['last_day_month'] = (pd.to_datetime(reporting_df['term_end'], format="%d", unit='d') + pd.offsets.MonthEnd(n=1)).dt.day

reporting_df["active_sub_month_end"][reporting_df.last_day_term_end < reporting_df.last_day_month] = 0

In [77]:
## fix those one line transactions that still have wrong product_term_length
trx_id_counts = reporting_df['transaction_id'].value_counts(sort=False)
check_length_mask = reporting_df[reporting_df['transaction_id'].isin(trx_id_counts.index[trx_id_counts == 1])].index.to_list()

reporting_df["product_term_length"].loc[check_length_mask] = reporting_df['term_end'].dt.day - reporting_df.term_start.dt.day

In [78]:
## calculate total_booking_net columns
reporting_df_nocalcs["total_booking_net_chf"] = (
        reporting_df_nocalcs["new_booking_net_chf"] + reporting_df_nocalcs["renewal_booking_net_chf"]
)
reporting_df_nocalcs["total_booking_net_eur"] = (
        reporting_df_nocalcs["new_booking_net_eur"] + reporting_df_nocalcs["renewal_booking_net_eur"]
)

In [79]:
## adding vat_percentage to calculate sales_price
reporting_df['vat_eur_percentage'] = (reporting_df['vat_eur'] / reporting_df['total_booking_net_eur']).astype(float, errors='ignore')
reporting_df['vat_chf_percentage'] = (reporting_df['vat_chf'] / reporting_df['total_booking_net_chf']).astype(float, errors='ignore')

reporting_df['sales_price_eur'] = reporting_df['total_booking_net_eur'] * (
        1 + reporting_df['vat_eur_percentage'])
reporting_df['sales_price_chf'] = reporting_df['total_booking_net_chf'] * (
        1 + reporting_df['vat_chf_percentage'])

In [80]:
## append dataframes
reporting_df = reporting_df.append(reporting_df_nocalcs).reset_index(drop=True)

## convert revenue_month_date to date
reporting_df["revenue_month_date"] = pd.to_datetime(
    reporting_df["revenue_month_date"]
).dt.date

In [81]:
reporting_df

,transaction_id,zuid,payment_method,sku,type_of_transaction,country_name,transaction_date,term_start,term_end,product_group_finance,vat_eur,vat_chf,store_fees_chf,store_fees_eur,total_booking_net_chf,total_booking_net_eur,product_term_length_months,revenue_month_number,revenue_month_date,product_term_length,active_sub_month_end,active_sub_content,total_days,max_revenue_month_number,total_revenue_net_eur,total_revenue_net_chf,last_day_term_end,last_day_month,vat_eur_percentage,vat_chf_percentage,sales_price_eur,sales_price_chf,new_booking_net_chf,renewal_booking_net_chf,new_booking_net_eur,renewal_booking_net_eur
0,35777722_1,30284071,amazon,zattoo_amazon_firetv_cirkus_german,new_sale,Germany,2022-02-02 01:43:16,2022-02-02 01:43:16,2022-02-15 15:32:38,cirkus,0.341681,0.356865,0.227135,0.217470,1.342851,1.285714,1.0,1.0,2022-02-01,13.0,0.0,0.0,14.0,1.0,1.285714,1.342851,15.0,28.0,0.265752,0.265752,1.627395,1.699716,NaN,NaN,NaN,NaN
1,33935525_6,29365165,amazon,zattoo_amazon_firetv_cirkus_german,renewal,Germany,2022-02-17 06:03:22,2022-02-17 06:03:22,2022-03-20 06:03:21,cirkus,0.758403,0.792107,0.502941,0.481541,3.501946,3.352941,2.0,1.0,2022-02-01,12.0,0.0,0.0,31.0,2.0,1.297913,1.355592,20.0,31.0,0.226190,0.226190,4.111345,4.294053,NaN,NaN,NaN,NaN
2,33935525_6,29365165,amazon,zattoo_amazon_firetv_cirkus_german,renewal,Germany,2022-02-17 06:03:22,2022-02-17 06:03:22,2022-03-20 06:03:21,cirkus,0.000000,0.000000,0.502941,0.481541,0.000000,0.000000,2.0,2.0,2022-03-01,19.0,0.0,0.0,31.0,2.0,2.055028,2.146354,20.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36073433_1,27777727,amazon,zattoo_amazon_firetv_ultimate_german,new_sale,Germany,2022-02-25 18:21:53,2022-02-25 18:21:53,2022-03-28 18:21:53,base_ultimate,2.658403,2.776543,1.763445,1.688412,10.436940,9.992857,2.0,1.0,2022-02-01,4.0,0.0,0.0,31.0,2.0,1.289401,1.346702,28.0,31.0,0.266030,0.266030,12.651261,13.213483,NaN,NaN,NaN,NaN
4,36073433_1,27777727,amazon,zattoo_amazon_firetv_ultimate_german,new_sale,Germany,2022-02-25 18:21:53,2022-02-25 18:21:53,2022-03-28 18:21:53,base_ultimate,0.000000,0.000000,1.763445,1.688412,0.000000,0.000000,2.0,2.0,2022-03-01,27.0,0.0,0.0,31.0,2.0,8.703456,9.090238,28.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39293,35990352_1,30403967,amazon,zattoo_amazon_firetv_ultimate_german_freetrial,free_trial,Germany,2022-02-18 19:57:42,2022-02-18 19:57:42,2022-03-21 19:57:42,base_ultimate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
39294,35985778_1,26848642,amazon,zattoo_amazon_firetv_ultimate_german_freetrial,free_trial,Germany,2022-02-18 11:35:09,2022-02-18 11:35:09,2022-03-21 11:35:09,base_ultimate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
39295,35989551_1,29993902,amazon,zattoo_amazon_firetv_ultimate_german_freetrial,free_trial,Germany,2022-02-18 19:12:43,2022-02-18 19:12:43,2022-03-21 19:12:43,base_ultimate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
39296,35989316_1,30403087,amazon,zattoo_amazon_firetv_ultimate_german_freetrial,free_trial,Germany,2022-02-18 19:01:22,2022-02-18 19:01:22,2022-03-21 19:01:22,base_ultimate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [45]:
## reorder dataframe
reporting_df = reporting_df[
    [
        'vendor_trx_id',
        'transaction_date',
        'term_end',
        'reporting_month',
        'country_name',
        # 'country_code',
        'currency',
        # 'vendor_sku',
        'sku',
        'subscription_status',
        'transaction_type',
        'payment_method',
        'product_class',
        # 'term',
        'product_length',
        'product_length_months',
        'product_term_length',
        # 'product_term_length_months',
        'domestic_abroad',
        'vat_percentage',
        'exchange_rate_eur_to_chf',
        'store_fees',
        'units',
        # 'sales_price',
        # 'earnings',
        # 'avg_price_sales_per_sub',
        'charge_eur',
        'sales_price_eur',
        'fee_eur',
        'vat_eur',
        'net_booking_eur',
        'payout_eur',
        'total_revenue_net_eur',
        'charge_chf',
        'sales_price_chf',
        'fee_chf',
        'vat_chf',
        'net_booking_chf',
        'payout_chf',
        'total_revenue_net_chf',
        'revenue_month_number',
        'revenue_month_date',
        # 'total_days',
        'active_sub_month_end',
        'active_sub_content'
    ]
]

## define BQ table schema
bq_schema = [
    {"name": "vendor_trx_id", "type": "STRING"},
    {"name": "transaction_date", "type": "TIMESTAMP"},
    {"name": "term_end", "type": "TIMESTAMP"},
    {"name": "reporting_month", "type": "STRING"},
    {"name": "country_name", "type": "STRING"},
    {"name": "currency", "type": "STRING"},
    {"name": "sku", "type": "STRING"},
    {"name": "subscription_status", "type": "STRING"},
    {"name": "transaction_type", "type": "STRING"},
    {"name": "payment_method", "type": "STRING"},
    {"name": "product_class", "type": "STRING"},
    {"name": "product_length", "type": "INTEGER"},
    {"name": "product_length_months", "type": "INTEGER"},
    {"name": "product_term_length", "type": "INTEGER"},
    {"name": "product_term_length", "type": "INTEGER"},
    {"name": "domestic_abroad", "type": "STRING"},
    {"name": "vat_percentage", "type": "FLOAT"},
    {"name": "exchange_rate_eur_to_chf", "type": "FLOAT"},
    {"name": "store_fees", "type": "INTEGER"},
    {"name": "units", "type": "INTEGER"},
    {"name": "charge_eur", "type": "FLOAT"},
    {"name": "sales_price_eur", "type": "FLOAT"},
    {"name": "fee_eur", "type": "FLOAT"},
    {"name": "vat_eur", "type": "FLOAT"},
    {"name": "net_booking_eur", "type": "FLOAT"},
    {"name": "payout_eur", "type": "FLOAT"},
    {"name": "total_revenue_net_eur", "type": "FLOAT"},
    {"name": "charge_chf", "type": "FLOAT"},
    {"name": "sales_price_chf", "type": "FLOAT"},
    {"name": "fee_chf", "type": "FLOAT"},
    {"name": "vat_chf", "type": "FLOAT"},
    {"name": "net_booking_chf", "type": "FLOAT"},
    {"name": "payout_chf", "type": "FLOAT"},
    {"name": "total_revenue_net_chf", "type": "FLOAT"},
    {"name": "revenue_month_number", "type": "INTEGER"},
    {"name": "revenue_month_date", "type": "TIMESTAMP"},
    {"name": "active_sub_month_end", "type": "INTEGER"},
    {"name": "active_sub_content", "type": "INTEGER"}
]

KeyError: "['charge_eur', 'units', 'product_length_months', 'product_class', 'vat_percentage', 'store_fees', 'net_booking_eur', 'transaction_type', 'reporting_month', 'currency', 'fee_eur', 'payout_eur', 'fee_chf', 'charge_chf', 'exchange_rate_eur_to_chf', 'vendor_trx_id', 'domestic_abroad', 'net_booking_chf', 'product_length', 'payout_chf', 'subscription_status'] not in index"

In [36]:
## export to csv
path = r"/Users/miguelcouto/Desktop/"

reporting_df.to_csv(os.path.join(path, r'f"subs_reporting_datatrans{reporting_month.replace("-", "_")}".csv'))

In [34]:
## export to BQ table
pandas_gbq.to_gbq(
    dataframe=reporting_df,
    destination_table=f"temp.subs_reporting_datatrans{reporting_month.replace('-', '')}",
    project_id="zattoo-dataeng",
    if_exists="replace",
    progress_bar=None,
    table_schema=bq_schema,
)